In [1]:
import numpy
import pandas as pd
import numpy as np
import scipy
from Utils import ut

In [6]:
#url = 'http://island.ricerca.di.unimi.it/~alfio/shared/worldcities.xlsx'
df= pd.read_excel(r"C:\Users\cibei\OneDrive\Desktop\Coding_for_data_science\Python\dscoding-projects\niccolo.cibei\travel_project\travel_project\dataset\worldcities.xlsx")
df = df.drop(index= range(100, 26569), axis=0)

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140
...,...,...,...,...,...,...,...,...,...,...,...
95,Nanchong,Nanchong,30.7991,106.0784,China,CN,CHN,Sichuan,minor,6183000.0,1156762337
96,Zunyi,Zunyi,27.7050,106.9336,China,CN,CHN,Guizhou,NaN,6127009.0,1156539782
97,Lu’an,Lu'an,31.7542,116.5078,China,CN,CHN,Anhui,minor,6090000.0,1156499624
98,Yichun,Yichun,27.8041,114.3830,China,CN,CHN,Jiangxi,minor,6048700.0,1156229590


In [8]:
#works!!!
import networkx as nx
import pandas as pd
from haversine import haversine

# Create a directed graph
G = nx.DiGraph()

# Add nodes for each city using their IDs
for city_id in df['id']:
    G.add_node(city_id)

# Calculate distances and add edges between cities
for i, row in df.iterrows():
    city_id = row['id']
    for j in range(1, 31):  # Iterate over closest_city_1, closest_city_2, closest_city_3, ...
        closest_id = row[f'closest_city_{j}']
        if closest_id is not None:  # we want to be sure that the closest city exists
            coords1 = (row['lat'], row['lng'])
            coords2 = df.loc[df['id'] == closest_id, ['lat', 'lng']].values[0]
            distance = haversine(coords1, coords2)  # Calculate the Haversine distance
            G.add_edge(city_id, closest_id, weight=distance)

# Input the source and target city names
source_city_name = "London"  # Replace with the actual source city name
target_city_name = "New York"  # Replace with the actual target city name

# Find the corresponding IDs for the source and target cities
source_id = df.loc[df['city_ascii'] == source_city_name, 'id'].values[0]
target_id = df.loc[df['city_ascii'] == target_city_name, 'id'].values[0]

# Find the shortest path between the source and target city IDs
shortest_path = nx.shortest_path(G, source=source_id, target=target_id, weight='weight')

print("Shortest Path (City IDs):", shortest_path)


NetworkXNoPath: No path between 1826645935 and 1840034016.

In [5]:
#works!!!
import folium

def create_map(df, shortest_path):
    # Create a map centered at the midpoint of the shortest path
    midpoint_idx = len(shortest_path) // 2
    midpoint_coords = df.loc[df['id'] == shortest_path[midpoint_idx], ['lat', 'lng']].values[0]
    mappa = folium.Map(location=midpoint_coords, zoom_start=6)

    # Add markers for each city along the shortest path
    for city_id in shortest_path:
        city_data = df.loc[df['id'] == city_id]
        if not city_data.empty:
            city_coords = city_data[['lat', 'lng']].values[0]
            city_name = city_data['city_ascii'].values[0]
            folium.Marker(city_coords, popup=city_name).add_to(mappa)

    # Create a PolyLine to connect the cities in the shortest path
    locations = [df.loc[df['id'] == city_id, ['lat', 'lng']].values[0] for city_id in shortest_path if not df.loc[df['id'] == city_id].empty]
    line = folium.PolyLine(locations=locations, color='blue', weight=5, opacity=0.7)
    line.add_to(mappa)

    # Save the map to an HTML file
    mappa.save('shortest_path_map.html')

# Input the source and target city names
source_city_name = "London"  # Replace with the actual source city name
target_city_name = "Tokyo"  # Replace with the actual target city name

# Find the corresponding IDs for the source and target cities
source_id = df.loc[df['city_ascii'] == source_city_name, 'id'].values
target_id = df.loc[df['city_ascii'] == target_city_name, 'id'].values

# Ensure that the source and target IDs are not empty

source_id = source_id[0]
target_id = target_id[0]

# Find the shortest path between the source and target city IDs
shortest_path = nx.shortest_path(G, source=source_id, target=target_id, weight='weight')

# Create and save the map for the shortest path
create_map(df, shortest_path)


In [ ]:
import networkx as nx

from haversine import haversine



# Create a directed graph
G = nx.DiGraph()

# Add nodes for each city using their IDs
for city_id in df['id']:
    G.add_node(city_id)

# Calculate distances and add edges between cities
for i, row in df.iterrows():
    city_id = row['id']
    for j in range(1, 4):  # Assuming 3 closest cities
        closest_id = row[f'closest_city_{j}']
        if closest_id is not None:
            coords1 = (row['lat'], row['lng'])
            coords2 = df.loc[df['id'] == closest_id, ['lat', 'lng']].values[0]
            distance = haversine(coords1, coords2)  # Calculate the Haversine distance
            G.add_edge(city_id, closest_id, weight=distance)

# Start from London
source_city_name = "London"
source_id = df.loc[df['city_ascii'] == source_city_name, 'id'].values[0]

# Find the shortest path that starts from London and ends in London
shortest_path = nx.shortest_path(G, source=source_id, target=source_id, weight='weight')

print("Shortest Path (City IDs):", shortest_path)


In [7]:
hav_matrix = ut.haversine_matrix(df)
hav_matrix

id,1392685764,1360771077,1356872604,1356226629,1608618140,1156073548,1076532519,1410836482,1484247881,1156237133,...,1156932948,1156829655,1156275243,1840015149,1156477539,1156762337,1156539782,1156499624,1156229590,1156322307
id,,,,,,,,,,,,,,,,,,,,,
1392685764,NaN,103.330660,121.981129,133.747195,54.726319,36.441739,379.438080,24.903124,464.549213,56.794950,...,35.910779,45.989732,39.333268,427.182962,74.691240,65.918513,65.433672,45.635730,51.444760,37.990805
1360771077,103.330660,NaN,88.792425,82.129171,48.819357,77.898765,299.750476,93.567289,402.831674,58.291842,...,89.150134,64.565794,75.342680,368.292266,61.192340,71.848646,65.828679,76.035505,67.622079,73.353157
1356872604,121.981129,88.792425,NaN,20.656595,89.178857,85.985223,260.865283,98.124945,342.730626,70.739239,...,87.124927,80.585973,83.383437,305.587850,49.936083,56.138085,57.674041,76.460165,72.120045,85.761236
1356226629,133.747195,82.129171,20.656595,NaN,93.816894,97.305534,246.096103,111.154042,333.734584,78.868085,...,100.569045,89.537825,94.433766,297.302195,59.160746,68.482342,68.315804,88.315905,82.438100,96.152312
1608618140,54.726319,48.819357,89.178857,93.816894,NaN,32.173509,333.602108,46.118016,427.270283,22.329309,...,44.259549,20.555313,30.421034,391.044167,40.846095,42.718339,37.283321,34.410737,28.650500,27.305033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156762337,65.918513,71.848646,56.138085,68.482342,42.718339,29.871576,314.578444,42.682218,398.857041,20.390591,...,32.137431,26.825453,27.362579,361.651593,12.946337,NaN,6.229913,20.324329,17.132384,30.065060
1156539782,65.433672,65.828679,57.674041,68.315804,37.283321,28.992656,314.190590,43.404550,400.229862,15.151282,...,33.580406,23.252653,26.152923,363.200840,9.694912,6.229913,NaN,20.173579,14.457964,28.177891
1156499624,45.635730,76.035505,76.460165,88.315905,34.410737,9.690704,334.306413,23.292877,419.168651,17.887450,...,14.328960,13.874109,7.678689,381.935683,29.782866,20.324329,20.173579,NaN,8.704664,11.271708


In [33]:
import networkx as nx

G = nx.DiGraph()

# Add nodes for each city using their IDs
for city_id in df['id']:
    G.add_node(city_id)
    
cc = pd.DataFrame(hav_matrix.iloc[0].sort_values())
for i in range(1,4):
   G.add_edge(df.iloc[11][0], cc['id'], weight=distance)

KeyError: 1

In [38]:

cc = pd.DataFrame(hav_matrix.iloc[0].sort_values())
cc.iloc[0]

KeyError: 0

In [15]:
data = hav_matrix 
for i in range(len(data)):
    data.iloc[i].sort_values()

id,1392685764,1360771077,1356872604,1356226629,1608618140,1156073548,1076532519,1410836482,1484247881,1156237133,...,1156932948,1156829655,1156275243,1840015149,1156477539,1156762337,1156539782,1156499624,1156229590,1156322307
id,,,,,,,,,,,,,,,,,,,,,
1392685764,NaN,103.330660,121.981129,133.747195,54.726319,36.441739,379.438080,24.903124,464.549213,56.794950,...,35.910779,45.989732,39.333268,427.182962,74.691240,65.918513,65.433672,45.635730,51.444760,37.990805
1360771077,103.330660,NaN,88.792425,82.129171,48.819357,77.898765,299.750476,93.567289,402.831674,58.291842,...,89.150134,64.565794,75.342680,368.292266,61.192340,71.848646,65.828679,76.035505,67.622079,73.353157
1356872604,121.981129,88.792425,NaN,20.656595,89.178857,85.985223,260.865283,98.124945,342.730626,70.739239,...,87.124927,80.585973,83.383437,305.587850,49.936083,56.138085,57.674041,76.460165,72.120045,85.761236
1356226629,133.747195,82.129171,20.656595,NaN,93.816894,97.305534,246.096103,111.154042,333.734584,78.868085,...,100.569045,89.537825,94.433766,297.302195,59.160746,68.482342,68.315804,88.315905,82.438100,96.152312
1608618140,54.726319,48.819357,89.178857,93.816894,NaN,32.173509,333.602108,46.118016,427.270283,22.329309,...,44.259549,20.555313,30.421034,391.044167,40.846095,42.718339,37.283321,34.410737,28.650500,27.305033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156762337,65.918513,71.848646,56.138085,68.482342,42.718339,29.871576,314.578444,42.682218,398.857041,20.390591,...,32.137431,26.825453,27.362579,361.651593,12.946337,NaN,6.229913,20.324329,17.132384,30.065060
1156539782,65.433672,65.828679,57.674041,68.315804,37.283321,28.992656,314.190590,43.404550,400.229862,15.151282,...,33.580406,23.252653,26.152923,363.200840,9.694912,6.229913,NaN,20.173579,14.457964,28.177891
1156499624,45.635730,76.035505,76.460165,88.315905,34.410737,9.690704,334.306413,23.292877,419.168651,17.887450,...,14.328960,13.874109,7.678689,381.935683,29.782866,20.324329,20.173579,NaN,8.704664,11.271708
